# Power Production - a Machine Learning project playground

>Author: **Andrzej Kocielski**  

This is a playground notebook for testing only. The actual project notebook is [Powerproduction_ML.ipynb](https://github.com/andkoc001/Machine-Learning-and-Statistics-Project.git/Powerproduction_ML.ipynb).

For more information see [README.md](https://github.com/andkoc001/Machine-Learning-and-Statistics-Project.git/README.MD).


___

## Loose notes and ideas

### Ideas

1. There is a number of observation in the data set where produced power output is zero, regardless of the wind speed. These data points should be removed from analysis.
2. ...

### ML Techniques

- Unsupervised
    - Clustering
    - Dimensional reduction
- Supervised
    - Regression
    - Classification
- Reinforced Learning

![image](https://miro.medium.com/max/700/1*AqNYz4M_GgfUN2ROb798yg.jpeg)

Image source: [Medium.com](https://miro.medium.com/max/700/1*AqNYz4M_GgfUN2ROb798yg.jpeg)

### ML algorithms

- Linear regression
- Logarithmic regression
- Decision Tree
- Decision Forrest
- Random Forrest
- t-Test
- k nearest neighbour (kNN)
- k-means
- Anova (analysis of variance)
- Support Vector Machine (SVM)
- Principal Component Analysis (PCA)
- Naive Bayes
- Dimensionality Reduction Algorithms

![image](https://docs.microsoft.com/en-us/azure/machine-learning/media/algorithm-cheat-sheet/machine-learning-algorithm-cheat-sheet.svg)
Image source: [Microsoft.com](https://docs.microsoft.com/en-us/azure/machine-learning/media/algorithm-cheat-sheet/machine-learning-algorithm-cheat-sheet.svg)

![image](https://miro.medium.com/max/1920/1*Lejtm0oGlOC5U0-J0JmGhg.png)
Image source: [Medium.com](https://miro.medium.com/max/1920/1*Lejtm0oGlOC5U0-J0JmGhg.png)

### To Do

1. Exploratory data analysis
2. Data cleaning
3. Data modeling (add / combine / infer additional data)
4. Select ML techniques to be used (explain why)
5. Do ML - analyse predictions accuracy etc. for various boundary conditions and parameters
6. Draw a conclusion

___

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests

## Data set

In [ ]:
# Data set is loaded from the file `powerproduction.txt`.
df = pd.read_csv(r"powerproduction.txt")

In [ ]:
df

## Import Seaborn

In [ ]:
# plotting config
plt.style.use('ggplot')
plt.figure(figsize=(12,6))

In [ ]:
sns.relplot(data=df, x="speed", y="power", s=10, height=4, aspect=2)
plt.plot([8,8],[0,120], "k:")
plt.plot([17,17],[0,120], "k:")
plt.plot([24.5,24.5],[0,120], "k:")
plt.text(3, 85, "A", size='20', color='black')
plt.text(12, 85, "B", size='20', color='black')
plt.text(20.5, 55, "C", size='20', color='black')
plt.text(25.2, 55, "F", size='20', color='black')

In [ ]:
# a new dataframe, showing how speed values are separated from each other - difference between subsequent and current speed value
df_sd = pd.DataFrame()
print(df_sd)
for index, row in df.iterrows():
    speed_difference = df.iloc[index]['speed'] - df.iloc[index-1]['speed']
    # print(speed_difference)
    df_sd = df_sd.append({'speed diff': speed_difference}, ignore_index=True)

df_sd = df_sd.drop(index=[0], axis=0)
print(df_sd)  

In [ ]:
df_sd.describe()

In [ ]:
sns.distplot(df_sd)

In [ ]:
# what wind speeds dominate - it appears to be more or less uniformely distributed
plt.figure(figsize=(20,2))
sns.distplot(df.speed, bins=100, kde=False)

In [ ]:
plt.figure(figsize=(20,4))
sns.distplot(df.power, bins=100, kde=False)

In [ ]:
sns.relplot(
    data=df,
    x="speed", y="power",
    kind="line", size_order=["T1", "T2"], palette="pastel",
    height=6, aspect=3, facet_kws=dict(sharex=False)
)

In [ ]:
# Linear regression is an underfitting approximation
plt.figure(figsize=(15,4))
sns.regplot(data=df, x="speed", y="power", scatter_kws={'s':1})

In [ ]:
# clean the dataset by removing all observations where the power output is zero

df_clean = df[df['power'] !=0]
df_clean

In [ ]:
sns.jointplot(x = "speed", y = "power", data = df_clean)

In [ ]:
ord = np.polyfit(df_clean["speed"], df_clean["power"], 3)
print(ord)

# y = pow(df_clean["speed"],ord[0]) + pow(df_clean["speed"],ord[1]) + ord[2]

x = df_clean["speed"]
# plt.ylim([0, 110])
plt.plot(x, df_clean["power"], "b.")
plt.plot(x, ord[0]*pow(x,3) + ord[1]*pow(x,2) + ord[2]*pow(x,1) + ord[3], "r-")


In [ ]:
# Do regression on the x and y arrays using numpy.

coefficients = []
for order in range(9):
    order = order
    coefficients = np.polyfit(df_clean["speed"], df_clean["power"], order)
    print(f"Order {order+1}: ", end="")
    for i in range(order+1):
        print(f"{coefficients[i]:.6f} ", end="  ")
        y = pow(df_clean["speed"],i) 
        
    print("\n")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x=df_clean["speed"], y=df_clean["power"], test_size=0.3, random_state=101)

In [ ]:
# Polynomial regression for cleaned dataset
a_plot = sns.lmplot(data=df_clean, x="speed", y="power", order=9, height=6, aspect=2, scatter_kws={'s':1})

a_plot.set(xlim=(0, 25))
a_plot.set(ylim=(0, 120))

plt.show()

The above polynomial appears to closely follow the pattern of the data points in the domain (wind speed in range 0-25).

Let's now apply the Numpy function `polyfit()` to get the value of the coefficients that minimise the squared order.

In [ ]:
coeff = np.polyfit(df['speed'], df['power'], 9)
#coeff

Testing the above - attempt to reproduce the plot of the polynomial with the above coefficients.

In [ ]:
coeff = np.polyfit(df['speed'], df['power'], 9)
#coeff

#with warnings.catch_warnings():
#    warnings.simplefilter('ignore', np.RankWarning)    
#    y = np.poly1d(coeff)
    
yp = np.poly1d(coeff)
    
x = np.linspace(0, 24.5, 101)
xp = plt.plot(x, yp(x))

plt.xlim(0,25)
plt.ylim(0,120)

plt.rcParams['figure.figsize'] = [15, 6]
plt.show()
print("y = ")
print(yp)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
# from sklearn import linear_model

In [ ]:
# adapted from https://stackoverflow.com/q/51732577
# create a shorthand for the column names
X = df.iloc[:, 0].values.reshape(-1,1)
y = df.iloc[:, 1].values.reshape(-1,1)
plt.scatter(X, y, s=2, color='blue', label="data")

# Fitting Polynomial Regression to the dataset
poly_reg = PolynomialFeatures(degree = 2)
X_poly = poly_reg.fit_transform(X)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
plt.plot(X, lin_reg.predict(X_poly), ls="--", color = 'cyan', label='1st polynomial order')

poly_reg = PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
plt.plot(X, lin_reg.predict(X_poly), ls="--", color = 'green', label='2nd polynomial order')

poly_reg = PolynomialFeatures(degree = 10)
X_poly = poly_reg.fit_transform(X)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
plt.plot(X, lin_reg.predict(X_poly), ls="--", color = 'red', label='9th polynomial order')


# Visualising the Polynomial Regression results
plt.legend(loc='best')
# plt.xlim(0,25)
# plt.ylim(0,120)
plt.figure(figsize=(6,3))
plt.show()

In [ ]:
# adapted from https://stackoverflow.com/q/51732577
# create a shorthand for the column names
X = df.iloc[:, 0].values.reshape(-1,1)
y = df.iloc[:, 1].values.reshape(-1,1)

# Fitting Polynomial Regression to the dataset
poly_reg_2 = PolynomialFeatures(degree = 2)
X_poly = poly_reg_2.fit_transform(X)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y)

poly_reg_3 = PolynomialFeatures(degree = 3)
X_poly = poly_reg_3.fit_transform(X)
lin_reg_3 = LinearRegression()
lin_reg_3.fit(X_poly, y)

poly_reg_10 = PolynomialFeatures(degree = 10)
X_poly = poly_reg_10.fit_transform(X)
lin_reg_10 = LinearRegression()
lin_reg_10.fit(X_poly, y)


# Visualising the Polynomial Regression results
plt.scatter(X, y, s=2, color='blue', label="data")
plt.plot(X, lin_reg_2.predict(poly_reg_2.fit_transform(X)), ls="--", color = 'cyan', label='1st polynomial order')
plt.plot(X, lin_reg_3.predict(poly_reg_3.fit_transform(X)), ls="--", color = 'green', label='3rd polynomial order')
plt.plot(X, lin_reg_9.predict(poly_reg_9.fit_transform(X)), ls="--", color = 'red', label='9th polynomial order')
plt.legend(loc='best')
plt.xlim(0,25)
plt.ylim(0,120)
plt.show()

In [ ]:
# adapted from https://scipy-lectures.org/packages/scikit-learn/auto_examples/plot_polynomial_regression.html
X = df.iloc[:, 0]
y = df.iloc[:, 1].values.reshape(-1,1)
plt.scatter(X, y, s=2, color='blue', label="data")

X_test = np.array([X**i for i in range(2)]).T
regr = linear_model.LinearRegression()
regr.fit(X_test, y)
plt.plot(X, regr.predict(X_test), ls="--", color = 'cyan', label='1st polynomial order')

X_test = np.array([X**i for i in range(4)]).T
regr = linear_model.LinearRegression()
regr.fit(X_test, y)
plt.plot(X, regr.predict(X_test), ls="--", color = 'green', label='3rd polynomial order')

X_test = np.array([X**i for i in range(10)]).T
regr = linear_model.LinearRegression()
regr.fit(X_test, y)
plt.plot(X, regr.predict(X_test), ls="--", color="red", label='9th polynomial order')

plt.xlim(0,25)
plt.ylim(0,120)
plt.legend(loc='best')

In [ ]:
from scipy.stats import linregress

#help(linregress)
linregress(df)

Where (from the function help file):

slope - Slope of the regression line.

intercept - Intercept of the regression line.

rvalue - Correlation coefficient.

pvalue - Two-sided p-value for a hypothesis test whose null hypothesis is that the slope is zero, using Wald Test with t-distribution of the test statistic.

stderr - Standard error of the estimated gradient.

https://towardsdatascience.com/polynomial-regression-bbe8b9d97491

In [ ]:
from sklearn.model_selection import train_test_split

# assign "speed" and "power" sets to variables X and y
X, y = df["speed"], df["power"]

# random_state (seed) is set for consistancy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)

# convert the array shape and unify the lengths
X_train = X_train.values.reshape(-1,1)
y_train = y_train.values.reshape(-1,1)

# create an instance of a LinearRegression() model named lin_reg_model.
lin_reg_model = LinearRegression()

#Train/fit lin_reg_model on the training data.
lin_reg_model.fit(X_train, y_train)

In [ ]:
# reschape X_test
X_test = X_test.values.reshape(-1,1)

y_predictions = lin_reg_model.predict(X_test)

In [ ]:
from sklearn import metrics
from math import sqrt

y_predictions = y_predictions.flatten()
print('Mean absolute error (MAE):\t', 
      metrics.mean_absolute_error(y_test, y_predictions), '\t',
      (1./len(y_test))*(sum(abs(y_test-y_predictions))))
print('Mean squared error (MSE):\t', 
      metrics.mean_squared_error(y_test, y_predictions), '\t',
      (1./len(y_test))*(sum((y_test-y_predictions)**2)))
print('Root mean square error (RMSE):\t', 
      metrics.mean_squared_error(y_test, y_predictions, squared=False), '\t',
      sqrt((1./len(y_test))*(sum((y_test-y_predictions)**2))))

---

## Playing with Requests

The `requests` library has been now added to the other imported libraries on top of the notebook.

In [ ]:
url = "https://www.gmit.ie"

# also check https://www.httpbin.org

In [ ]:
res = requests.get(url)
res

In [ ]:
print(dir(res))
print(help(res))

In [ ]:
res.status_code

In [ ]:
print(res.headers)

In [ ]:
print(res.text) # print() is used for better text formating

# Test linear regression script to be put into a separate file

In [ ]:
# import libraries and packages - see description above
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


def lin_reg(wind_speed):
    '''
    Doc string to be added here
    '''

    # load the data set from file
    df_raw = pd.read_csv(r"powerproduction.txt")

    # clean the dataset by removing all observations where the power output is zero
    df = df_raw[df_raw['power'] !=0]

    # assign "speed" and "power" sets to variables X and y
    X, y = df["speed"], df["power"]

    # random_state (seed) is set for consistancy
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2020)

    # convert the array shape and unify the lengths
    X_train = X_train.values.reshape(-1,1)
    y_train = y_train.values.reshape(-1,1)
    X_test = X_test.values.reshape(-1,1)

    # create an instance of a LinearRegression() model named lin_reg_model.
    lin_reg_model = LinearRegression()

    #Train/fit lin_reg_model on the training data.
    lin_reg_model.fit(X_train, y_train)

    # input wind speed for prediction
    test = np.array([[wind_speed]])
    # test.shape

    # define prediction
    # prediction = lin_reg_model.predict(X_test).values.reshape(-1,1)
    prediction = lin_reg_model.predict(test)
    return float(prediction)
    
lin_reg(20)

# Test poly (7th order) regression script to be put into a separate file

In [6]:
# import libraries and packages - see description above
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures


def poly_reg(wind_speed):
    '''
    Doc string to be added here
    '''

    # load the data set from file
    df_raw = pd.read_csv(r"powerproduction.txt")

    # clean the dataset by removing all observations where the power output is zero
    df = df_raw[df_raw['power'] !=0]
    
    # create a shorthand for the column names
    X = df.iloc[:, 0].values.reshape(-1,1)
    y = df.iloc[:, 1]
        
    # develop a regression model
    poly = PolynomialFeatures(degree = 7) # 7th polynomial order
    X_poly = poly.fit_transform(X)
    # ask our model to fit the data.
    poly_reg = LinearRegression().fit(X_poly, y)
    # perform regression to predict the power output out of wind speed
    y_pred = poly_reg.predict(X_poly)
    
    # input wind speed for prediction
    x = np.array([[wind_speed]])
    
    # define prediction
    prediction = (-5.11800967e-06*pow(x,7)) + 4.48301902e-04*pow(x,6) - 1.52309426e-02*pow(x,5) + 2.50368085e-01*pow(x,4) - 2.04365136e+00*pow(x,3) + 8.13376871e+00*pow(x,2) - 1.38470256e+01*pow(x,1) + 10.91407191*pow(x,0)
    
    wind_speed = np.array([wind_speed]).reshape((-1, 1))
    print(wind_speed.shape)
    a = PolynomialFeatures(degree=7, include_bias=False).fit_transform(wind_speed)
    print(a.shape)
    print(a)
    y_pred = poly_reg.predict(a)
    print('predicted response: ', y_pred)
    
    return float(prediction)


poly_reg(20)

(1, 1)
(1, 7)
[[1.e+01 1.e+02 1.e+03 1.e+04 1.e+05 1.e+06 1.e+07]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 8 is different from 7)